![image](https://github.com/eWaterCycle/ewatercycle/raw/main/docs/examples/logo.png)

# Running LISFLOOD model using eWaterCycle package (on Cartesius machine of SURFsara)

This notebooks shows how to run [LISFLOOD](https://ec-jrc.github.io/lisflood-model/) model. Please note that the [lisflood-grpc4bmi docker image](https://hub.docker.com/r/ewatercycle/lisflood-grpc4bmi) in eWaterCycle is compatible only with forcing data and parameter set on Cartesius machine of SURFsara. More information about data, configuration and installation instructions can be found in the [System setup](https://ewatercycle.readthedocs.io/en/latest/system_setup.html) in the eWaterCycle documentation.

In [ ]:
import logging
logger = logging.getLogger('grpc4bmi.bmi_grpc_client')
logger.setLevel(logging.WARNING)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import ewatercycle.forcing
import ewatercycle.models
import ewatercycle.parameter_sets

## Load forcing data

For this example noetbook, lisflood_ERA-Interim_*_1990_1990.nc data are copied 
from `/projects/0/wtrcycle/comparison/forcing/lisflood` to `/scratch/shared/ewatercycle/lisflood_forcing_data`.
Also the lisvap output files 'e0', 'es0' and 'et0' are generated and store in the same directory.

In [ ]:
forcing = ewatercycle.forcing.load_foreign(target_model='lisflood',
                                           directory='/scratch/shared/ewatercycle/lisflood_forcing_data/',
                                           start_time='1990-01-01T00:00:00Z',
                                           end_time='1990-12-31T00:00:00Z',
                                           forcing_info={
                                               'PrefixPrecipitation': 'lisflood_ERA-Interim_pr_1990_1990.nc',
                                               'PrefixTavg': 'lisflood_ERA-Interim_tas_1990_1990.nc',
                                               'PrefixE0': 'lisflood_e0_1990_1990.nc',
                                               'PrefixES0': 'lisflood_es0_1990_1990.nc',
                                               'PrefixET0': 'lisflood_et0_1990_1990.nc',
                                           }
                                          )
print(forcing)

## Load parameter set

This example uses parameter set on Cartesius machine of SURFsara.

In [ ]:
parameterset = ewatercycle.parameter_sets.ParameterSet(
    name='Lisflood01degree_masked',
    directory='/projects/0/wtrcycle/comparison/lisflood_input/Lisflood01degree_masked',
    config='/projects/0/wtrcycle/comparison/lisflood_input/settings_templates/settings_lisflood.xml',
    target_model='lisflood'
)
print(parameterset)

## Set up the model

To create the model object, we need to select a version.

In [ ]:
ewatercycle.models.Lisflood.available_versions

In [ ]:
model = ewatercycle.models.Lisflood(version='20.10', parameter_set=parameterset, forcing=forcing)
print(model)

In [ ]:
model.parameters

Setup model with model_mask, IrrigationEfficiency of 0.8 and an earlier end time, making total model time just 1 month.

In [ ]:
model_mask = '/projects/0/wtrcycle/comparison/recipes_auxiliary_datasets/LISFLOOD/model_mask.nc'

config_file, config_dir = model.setup(IrrigationEfficiency='0.8', 
                                      end_time='1990-1-31T00:00:00Z'
                                      MaskMap=model_mask)
print(cfg_file)
print(cfg_dir)

In [ ]:
model.parameters

Initialize the model with the config file:

In [ ]:
model.initialize(config_file)

Get model variable names

In [ ]:
model.output_var_names

## Run the model

Store simulated values at one location until model end time.

In [ ]:
discharge_at_one_location = []
time_range = []
end_time = model.end_time

while (model.time < end_time):
    model.update()
    discharge_at_one_location.append(model.get_value_at_coords('Discharge', lon=[-71.35], lat=[42.64]))
    time_range.append(model.time_as_datetime.date())
    print(model.time_as_isostr)

In [ ]:
model.finalize()

Store simulated values for all locations of the model grid at end time.

In [ ]:
discharge = model.get_value_as_xarray('Discharge')

## Inspect the results

In [ ]:
simulated_discharge_at_one_location = pd.DataFrame(
    {'simulation': discharge_at_one_location}, index=pd.to_datetime(time_range)
)
simulated_discharge.plot(figsize=(12, 8))

In [ ]:
lc = discharge.coords["longitude"]
la = discharge.coords["latitude"]
discharge.loc[
    dict(longitude=lc[(lc > 3) & (lc < 12)], latitude=la[(la > 46) & (la < 52)])].plot(robust=True, cmap='GnBu')